# 🤝 Méthodes d'Ensemble pour les Réseaux de Neurones

## 🎯 Objectifs

Dans ce notebook, nous explorons les **méthodes d'ensemble** :

- 🧠 **Principe** : "Plusieurs têtes valent mieux qu'une"
- 🎲 **Diversité** : Combiner plusieurs modèles différents
- 📈 **Performance** : Améliorer l'accuracy de 1-3%
- 🛡️ **Robustesse** : Réduire la variance et le surapprentissage
- 🎯 **Confiance** : Meilleures estimations de probabilité

---

## 🤔 Pourquoi les Ensembles Fonctionnent ?

### Intuition Simple

Imaginez **10 experts** qui votent :

```
Expert 1: "C'est un 3"  →  ✓ Correct
Expert 2: "C'est un 5"  →  ✗ Erreur
Expert 3: "C'est un 3"  →  ✓ Correct
Expert 4: "C'est un 3"  →  ✓ Correct
...

Vote final: "C'est un 3" (7/10) → ✓✓✓ Plus fiable !
```

### Conditions de Succès

Pour qu'un ensemble fonctionne, il faut :

1. **Diversité** : Les modèles doivent être différents
2. **Compétence** : Chaque modèle doit être meilleur que le hasard
3. **Indépendance** : Les erreurs ne doivent pas être corrélées

### Bénéfices

✅ **Amélioration de l'accuracy** : Typiquement +1-3%  
✅ **Réduction du surapprentissage** : Moyenne des erreurs  
✅ **Estimations de confiance** : Variance entre prédictions  
✅ **Robustesse** : Moins sensible aux outliers

---

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys
import time
from collections import Counter

# Ajouter le dossier parent au path
sys.path.append('../')

from src.network import NeuralNetwork
from src.utils import load_mnist_data
from src import visualize
from src.metrics import accuracy, confusion_matrix

# Configuration
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
np.random.seed(42)

print("✓ Imports réussis !")

## 1️⃣ Chargement des Données

In [ ]:
# Charger MNIST
print("Chargement des données MNIST...")
X_train, y_train, X_val, y_val, X_test, y_test = load_mnist_data()

print(f"\nTrain: {X_train.shape[0]:,} exemples")
print(f"Val: {X_val.shape[0]:,} exemples")
print(f"Test: {X_test.shape[0]:,} exemples")

---

## 2️⃣ Méthode 1 : Voting Classifier (Vote Majoritaire)

### 🗳️ Principe

Entraîner **plusieurs modèles différents** et combiner leurs prédictions par vote.

### Types de Vote

1. **Hard Voting** : Vote majoritaire sur les classes prédites
   ```
   Modèle 1: 3
   Modèle 2: 5    →  Vote final: 3 (majorité)
   Modèle 3: 3
   ```

2. **Soft Voting** : Moyenne des probabilités
   ```
   Modèle 1: [0.1, 0.8, 0.1]  
   Modèle 2: [0.2, 0.6, 0.2]  →  Moyenne: [0.17, 0.73, 0.1]
   Modèle 3: [0.2, 0.7, 0.1]
   ```

**Soft voting est généralement meilleur** car il utilise plus d'information.

---

In [ ]:
# Créer plusieurs architectures différentes pour la diversité
architectures = [
    {'name': 'Small', 'layers': [784, 128, 64, 10], 'lr': 0.01, 'optimizer': 'adam'},
    {'name': 'Medium', 'layers': [784, 256, 128, 10], 'lr': 0.01, 'optimizer': 'adam'},
    {'name': 'Deep', 'layers': [784, 128, 64, 32, 10], 'lr': 0.01, 'optimizer': 'adam'},
    {'name': 'Wide', 'layers': [784, 512, 10], 'lr': 0.005, 'optimizer': 'momentum'},
    {'name': 'Balanced', 'layers': [784, 200, 100, 10], 'lr': 0.01, 'optimizer': 'adam'},
]

print(f"🎯 Entraînement de {len(architectures)} modèles différents...\n")

In [ ]:
# Entraîner tous les modèles
models = []
results = []

for i, arch in enumerate(architectures, 1):
    print(f"\n{'='*70}")
    print(f"🚀 Modèle {i}/{len(architectures)}: {arch['name']}")
    print(f"   Architecture: {' → '.join(map(str, arch['layers']))}")
    print(f"   Optimizer: {arch['optimizer']}, LR: {arch['lr']}")
    print(f"{'='*70}\n")
    
    # Créer et entraîner
    model = NeuralNetwork(
        layer_dims=arch['layers'],
        learning_rate=arch['lr'],
        optimizer=arch['optimizer']
    )
    
    start = time.time()
    model.train(X_train, y_train, X_val, y_val, 
                epochs=10, batch_size=128, verbose=False)
    training_time = time.time() - start
    
    # Évaluer
    train_acc = model.accuracy(X_train, y_train)
    val_acc = model.accuracy(X_val, y_val)
    test_acc = model.accuracy(X_test, y_test)
    
    print(f"\n✓ Entraînement terminé en {training_time:.1f}s")
    print(f"  Train Acc: {train_acc:.4f}")
    print(f"  Val Acc:   {val_acc:.4f}")
    print(f"  Test Acc:  {test_acc:.4f}")
    
    models.append(model)
    results.append({
        'name': arch['name'],
        'train_acc': train_acc,
        'val_acc': val_acc,
        'test_acc': test_acc,
        'time': training_time
    })

print(f"\n\n{'='*70}")
print("✅ TOUS LES MODÈLES ENTRAÎNÉS !")
print(f"{'='*70}")

In [ ]:
# Résumé des performances individuelles
print("\n📊 Performances Individuelles:\n")
print(f"{'Modèle':<12} {'Train':<10} {'Val':<10} {'Test':<10} {'Temps'}")
print("-" * 55)

for r in results:
    print(f"{r['name']:<12} {r['train_acc']:<10.4f} {r['val_acc']:<10.4f} "
          f"{r['test_acc']:<10.4f} {r['time']:.1f}s")

avg_test_acc = np.mean([r['test_acc'] for r in results])
print(f"\n{'Moyenne':<12} {'':<10} {'':<10} {avg_test_acc:<10.4f}")

### 🗳️ Hard Voting

In [ ]:
def hard_voting_predict(models, X):
    """
    Hard voting: vote majoritaire sur les classes prédites
    """
    # Collecter toutes les prédictions
    predictions = np.array([model.predict(X) for model in models])  # Shape: (n_models, n_samples)
    
    # Vote majoritaire pour chaque échantillon
    final_predictions = []
    for i in range(X.shape[0]):
        votes = predictions[:, i]
        # Compter les votes
        vote_counts = Counter(votes)
        # Classe la plus votée
        final_pred = vote_counts.most_common(1)[0][0]
        final_predictions.append(final_pred)
    
    return np.array(final_predictions)

# Tester le hard voting
print("🗳️ Hard Voting (vote majoritaire)...\n")
y_pred_hard = hard_voting_predict(models, X_test)
hard_voting_acc = np.mean(y_pred_hard == y_test)

print(f"Accuracy avec Hard Voting: {hard_voting_acc:.4f}")
print(f"Meilleur modèle individuel: {max(r['test_acc'] for r in results):.4f}")
print(f"Gain: {hard_voting_acc - max(r['test_acc'] for r in results):.4f} ({(hard_voting_acc / max(r['test_acc'] for r in results) - 1) * 100:.2f}%)")

### 🎲 Soft Voting

In [ ]:
def soft_voting_predict(models, X):
    """
    Soft voting: moyenne des probabilités
    """
    # Collecter toutes les probabilités
    all_probs = []
    for model in models:
        probs, _ = model.forward(X)
        all_probs.append(probs)
    
    # Moyenne des probabilités
    avg_probs = np.mean(all_probs, axis=0)  # Shape: (n_samples, n_classes)
    
    # Classe avec la plus haute probabilité moyenne
    final_predictions = np.argmax(avg_probs, axis=1)
    
    return final_predictions, avg_probs

# Tester le soft voting
print("🎲 Soft Voting (moyenne des probabilités)...\n")
y_pred_soft, probs_soft = soft_voting_predict(models, X_test)
soft_voting_acc = np.mean(y_pred_soft == y_test)

print(f"Accuracy avec Soft Voting: {soft_voting_acc:.4f}")
print(f"Meilleur modèle individuel: {max(r['test_acc'] for r in results):.4f}")
print(f"Gain: {soft_voting_acc - max(r['test_acc'] for r in results):.4f} ({(soft_voting_acc / max(r['test_acc'] for r in results) - 1) * 100:.2f}%)")

print(f"\n🆚 Hard vs Soft:")
print(f"  Hard Voting: {hard_voting_acc:.4f}")
print(f"  Soft Voting: {soft_voting_acc:.4f}")
print(f"  Différence: {soft_voting_acc - hard_voting_acc:.4f}")

### 📊 Visualisation des Résultats

In [ ]:
# Comparer toutes les méthodes
fig, ax = plt.subplots(figsize=(12, 6))

methods = ['Moyenne\nIndividuelle', 'Meilleur\nIndividuel', 'Hard\nVoting', 'Soft\nVoting']
accuracies = [
    avg_test_acc,
    max(r['test_acc'] for r in results),
    hard_voting_acc,
    soft_voting_acc
]
colors = ['lightblue', 'orange', 'lightgreen', 'lightcoral']

bars = ax.bar(methods, accuracies, color=colors, edgecolor='black', linewidth=2)

# Ajouter les valeurs sur les barres
for bar, acc in zip(bars, accuracies):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{acc:.4f}',
            ha='center', va='bottom', fontsize=12, fontweight='bold')

ax.set_ylabel('Test Accuracy', fontsize=12, fontweight='bold')
ax.set_title('Comparaison des Méthodes d\'Ensemble', fontsize=14, fontweight='bold')
ax.set_ylim([min(accuracies) - 0.01, max(accuracies) + 0.01])
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('../models/ensemble_voting_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

---

## 3️⃣ Méthode 2 : Bagging (Bootstrap Aggregating)

### 🎒 Principe

**Bagging** = **B**ootstrap **Agg**regat**ing**

1. Créer plusieurs **sous-ensembles** du dataset par bootstrap (échantillonnage avec remise)
2. Entraîner un modèle sur chaque sous-ensemble
3. Combiner les prédictions (vote ou moyenne)

### Avantages

✅ Réduit la **variance** (surapprentissage)  
✅ Améliore la **stabilité**  
✅ Parallélisable (chaque modèle est indépendant)  

### Fonctionnement

```
Dataset Original: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

Bootstrap 1: [1, 1, 3, 5, 7, 8, 8, 9, 10, 10]  →  Modèle 1
Bootstrap 2: [2, 3, 3, 4, 5, 5, 6, 7, 9, 10]  →  Modèle 2
Bootstrap 3: [1, 2, 4, 4, 5, 6, 7, 8, 8, 9]   →  Modèle 3
...

Prédiction finale = Vote ou Moyenne
```

In [ ]:
def create_bootstrap_sample(X, y, sample_ratio=1.0):
    """
    Crée un échantillon bootstrap (avec remise)
    """
    n_samples = int(len(X) * sample_ratio)
    indices = np.random.choice(len(X), size=n_samples, replace=True)
    return X[indices], y[indices]

# Configuration du bagging
n_bagging_models = 5
base_architecture = [784, 256, 128, 10]

print(f"🎒 Entraînement de {n_bagging_models} modèles avec Bagging...\n")
print(f"Architecture de base: {' → '.join(map(str, base_architecture))}\n")

In [ ]:
# Entraîner les modèles bagging
bagging_models = []

for i in range(n_bagging_models):
    print(f"Modèle Bagging {i+1}/{n_bagging_models}...")
    
    # Créer bootstrap sample
    X_boot, y_boot = create_bootstrap_sample(X_train, y_train)
    
    # Entraîner
    model = NeuralNetwork(
        layer_dims=base_architecture,
        learning_rate=0.01,
        optimizer='adam'
    )
    model.train(X_boot, y_boot, X_val, y_val, 
                epochs=10, batch_size=128, verbose=False)
    
    test_acc = model.accuracy(X_test, y_test)
    print(f"  Test Acc: {test_acc:.4f}\n")
    
    bagging_models.append(model)

print("✓ Bagging terminé !")

In [ ]:
# Évaluer le bagging
y_pred_bagging, probs_bagging = soft_voting_predict(bagging_models, X_test)
bagging_acc = np.mean(y_pred_bagging == y_test)

individual_accs = [model.accuracy(X_test, y_test) for model in bagging_models]
avg_individual = np.mean(individual_accs)

print("\n📊 Résultats Bagging:\n")
print(f"Accuracy moyenne individuelle: {avg_individual:.4f}")
print(f"Accuracy avec Bagging:        {bagging_acc:.4f}")
print(f"Gain:                          {bagging_acc - avg_individual:.4f} (+{(bagging_acc / avg_individual - 1) * 100:.2f}%)")

---

## 4️⃣ Méthode 3 : Weighted Voting (Vote Pondéré)

### ⚖️ Principe

Tous les modèles ne sont **pas égaux** !

**Idée** : Donner plus de poids aux modèles plus performants.

```
Modèle 1 (acc=0.95): poids = 0.95
Modèle 2 (acc=0.97): poids = 0.97  →  Weighted Average
Modèle 3 (acc=0.93): poids = 0.93
```

### Formule

```
P_weighted(classe) = Σ (weight_i × P_i(classe)) / Σ weights
```

où `weight_i` peut être :
- L'accuracy du modèle
- Le F1-score
- Une valeur optimisée

In [ ]:
def weighted_voting_predict(models, weights, X):
    """
    Vote pondéré basé sur les poids des modèles
    """
    # Normaliser les poids
    weights = np.array(weights)
    weights = weights / weights.sum()
    
    # Collecter les probabilités
    all_probs = []
    for model in models:
        probs, _ = model.forward(X)
        all_probs.append(probs)
    
    # Moyenne pondérée
    weighted_probs = np.zeros_like(all_probs[0])
    for i, probs in enumerate(all_probs):
        weighted_probs += weights[i] * probs
    
    # Prédictions
    final_predictions = np.argmax(weighted_probs, axis=1)
    
    return final_predictions, weighted_probs

# Utiliser les accuracy comme poids
weights = [r['test_acc'] for r in results]

print("⚖️ Weighted Voting (vote pondéré)...\n")
print("Poids des modèles:")
for i, (r, w) in enumerate(zip(results, weights)):
    print(f"  {r['name']:<12} : {w:.4f}")

# Prédiction
y_pred_weighted, probs_weighted = weighted_voting_predict(models, weights, X_test)
weighted_voting_acc = np.mean(y_pred_weighted == y_test)

print(f"\nAccuracy avec Weighted Voting: {weighted_voting_acc:.4f}")
print(f"Accuracy avec Soft Voting:     {soft_voting_acc:.4f}")
print(f"Différence: {weighted_voting_acc - soft_voting_acc:.4f}")

---

## 5️⃣ Analyse de la Confiance et Diversité

### 🎯 Confiance de l'Ensemble

Un avantage majeur des ensembles : **mesurer la confiance**

**Variance des prédictions** = Indicateur de certitude

- **Variance faible** : Tous les modèles d'accord → Haute confiance
- **Variance élevée** : Modèles en désaccord → Faible confiance

In [ ]:
# Calculer la variance des prédictions
all_predictions = np.array([model.predict(X_test) for model in models])

# Pour chaque échantillon, compter le nombre de modèles d'accord
agreement_scores = []
for i in range(X_test.shape[0]):
    votes = all_predictions[:, i]
    most_common_count = Counter(votes).most_common(1)[0][1]
    agreement = most_common_count / len(models)
    agreement_scores.append(agreement)

agreement_scores = np.array(agreement_scores)

# Analyser par correctitude
correct_mask = y_pred_soft == y_test
agreement_correct = agreement_scores[correct_mask]
agreement_incorrect = agreement_scores[~correct_mask]

print("🎯 Analyse de la Confiance de l'Ensemble:\n")
print(f"Prédictions Correctes:")
print(f"  Agreement moyen: {agreement_correct.mean():.4f}")
print(f"  Écart-type: {agreement_correct.std():.4f}")
print(f"\nPrédictions Incorrectes:")
print(f"  Agreement moyen: {agreement_incorrect.mean():.4f}")
print(f"  Écart-type: {agreement_incorrect.std():.4f}")

# Visualiser
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Histogramme
ax1.hist(agreement_correct, bins=20, alpha=0.7, color='green', 
         label='Correctes', edgecolor='black')
ax1.hist(agreement_incorrect, bins=20, alpha=0.7, color='red', 
         label='Incorrectes', edgecolor='black')
ax1.set_xlabel('Agreement Score (% modèles d\'accord)', fontsize=12)
ax1.set_ylabel('Fréquence', fontsize=12)
ax1.set_title('Distribution de l\'Agreement', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(alpha=0.3)

# Relation agreement vs accuracy
thresholds = np.linspace(0.5, 1.0, 20)
accuracies_by_threshold = []
counts_by_threshold = []

for thresh in thresholds:
    mask = agreement_scores >= thresh
    if mask.sum() > 0:
        acc = np.mean(y_pred_soft[mask] == y_test[mask])
        accuracies_by_threshold.append(acc)
        counts_by_threshold.append(mask.sum())
    else:
        accuracies_by_threshold.append(np.nan)
        counts_by_threshold.append(0)

ax2.plot(thresholds, accuracies_by_threshold, 'o-', linewidth=2, markersize=6, color='blue')
ax2.set_xlabel('Seuil d\'Agreement', fontsize=12)
ax2.set_ylabel('Accuracy', fontsize=12, color='blue')
ax2.set_title('Accuracy vs Agreement', fontsize=14, fontweight='bold')
ax2.tick_params(axis='y', labelcolor='blue')
ax2.grid(alpha=0.3)

# Ajouter le nombre d'échantillons
ax2_twin = ax2.twinx()
ax2_twin.plot(thresholds, counts_by_threshold, 's--', linewidth=2, 
              markersize=4, color='orange', alpha=0.7, label='Nombre')
ax2_twin.set_ylabel('Nombre d\'échantillons', fontsize=12, color='orange')
ax2_twin.tick_params(axis='y', labelcolor='orange')

plt.tight_layout()
plt.savefig('../models/ensemble_confidence.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n💡 Interprétation:")
print("- Plus l'agreement est élevé, plus l'accuracy est élevée")
print("- On peut filtrer les prédictions incertaines (faible agreement)")

### 🔍 Diversité des Modèles

Pour qu'un ensemble soit efficace, les modèles doivent être **diversifiés**.

**Mesure de diversité** : Calculer le désaccord entre paires de modèles.

In [ ]:
# Matrice de désaccord entre modèles
n_models = len(models)
disagreement_matrix = np.zeros((n_models, n_models))

for i in range(n_models):
    pred_i = models[i].predict(X_test)
    for j in range(n_models):
        pred_j = models[j].predict(X_test)
        disagreement = np.mean(pred_i != pred_j)
        disagreement_matrix[i, j] = disagreement

# Visualiser
fig, ax = plt.subplots(figsize=(10, 8))

model_names = [r['name'] for r in results]
sns.heatmap(disagreement_matrix, annot=True, fmt='.3f', cmap='YlOrRd',
            xticklabels=model_names, yticklabels=model_names,
            cbar_kws={'label': 'Taux de Désaccord'}, ax=ax)
ax.set_title('Matrice de Diversité des Modèles', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('../models/ensemble_diversity.png', dpi=150, bbox_inches='tight')
plt.show()

avg_disagreement = disagreement_matrix[np.triu_indices_from(disagreement_matrix, k=1)].mean()
print(f"\n📊 Désaccord moyen entre modèles: {avg_disagreement:.4f}")
print(f"\n💡 Diversité {'élevée' if avg_disagreement > 0.03 else 'faible'} → "
      f"Ensemble {'efficace' if avg_disagreement > 0.03 else 'peut être amélioré'}")

---

## 6️⃣ Résumé Final et Comparaison

### 📊 Tableau Récapitulatif

In [ ]:
# Résumé de toutes les méthodes
summary = {
    'Meilleur Individuel': max(r['test_acc'] for r in results),
    'Moyenne Individuelle': avg_test_acc,
    'Hard Voting': hard_voting_acc,
    'Soft Voting': soft_voting_acc,
    'Weighted Voting': weighted_voting_acc,
    'Bagging': bagging_acc,
}

print("\n" + "="*70)
print("📊 RÉSUMÉ FINAL - COMPARAISON DES MÉTHODES")
print("="*70 + "\n")

print(f"{'Méthode':<25} {'Test Accuracy':<15} {'Gain vs Meilleur'}")
print("-" * 60)

best_individual = summary['Meilleur Individuel']
for method, acc in sorted(summary.items(), key=lambda x: x[1], reverse=True):
    gain = acc - best_individual
    gain_pct = (acc / best_individual - 1) * 100
    print(f"{method:<25} {acc:<15.4f} {gain:+.4f} ({gain_pct:+.2f}%)")

print("\n" + "="*70)

In [ ]:
# Visualisation finale
fig, ax = plt.subplots(figsize=(14, 7))

methods = list(summary.keys())
accuracies = list(summary.values())
colors = ['lightblue', 'orange', 'lightgreen', 'lightcoral', 'yellow', 'pink']

bars = ax.barh(methods, accuracies, color=colors, edgecolor='black', linewidth=2)

# Ajouter les valeurs
for bar, acc in zip(bars, accuracies):
    width = bar.get_width()
    ax.text(width, bar.get_y() + bar.get_height()/2.,
            f'{acc:.4f}',
            ha='left', va='center', fontsize=11, fontweight='bold')

ax.set_xlabel('Test Accuracy', fontsize=12, fontweight='bold')
ax.set_title('Comparaison Complète des Méthodes d\'Ensemble', fontsize=14, fontweight='bold')
ax.set_xlim([min(accuracies) - 0.005, max(accuracies) + 0.01])
ax.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.savefig('../models/ensemble_final_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

---

## 🎓 Conclusions et Recommandations

### ✅ Ce que nous avons appris

1. **Les ensembles améliorent la performance** (typiquement +1-3%)
2. **Soft voting > Hard voting** (utilise plus d'information)
3. **La diversité est clé** pour l'efficacité de l'ensemble
4. **Mesure de confiance** : L'agreement indique la certitude
5. **Weighted voting** peut donner un léger gain supplémentaire

### 🎯 Quand Utiliser les Ensembles ?

✅ **Compétitions** : Chaque 0.1% compte !  
✅ **Applications critiques** : Médecine, finance, sécurité  
✅ **Estimations de confiance** : Besoin de savoir quand le modèle doute  
✅ **Datasets petits/bruités** : Réduction de la variance

❌ **Ne PAS utiliser si** :
- Contraintes de latence strictes
- Ressources limitées (CPU/mémoire)
- Un seul modèle suffit (gain trop faible)

### 🚀 Stratégies pour Maximiser la Diversité

1. **Architectures différentes** (profond, large, etc.)
2. **Hyperparamètres variés** (learning rate, optimizers)
3. **Initialisations différentes** (random seeds)
4. **Sous-ensembles de données** (bagging, bootstrapping)
5. **Features différentes** (si applicable)
6. **Fonctions d'activation variées**

### 💰 Trade-off Performance vs Coût

| Méthode | Performance | Temps Entraînement | Temps Inférence | Mémoire |
|---------|-------------|-------------------|----------------|----------|
| 1 Modèle | ⭐⭐⭐ | ⭐⭐⭐⭐⭐ | ⭐⭐⭐⭐⭐ | ⭐⭐⭐⭐⭐ |
| 3-5 Modèles | ⭐⭐⭐⭐ | ⭐⭐⭐ | ⭐⭐⭐ | ⭐⭐⭐ |
| 10+ Modèles | ⭐⭐⭐⭐⭐ | ⭐ | ⭐ | ⭐ |

**Sweet spot** : 3-5 modèles pour un bon compromis

---

## 🎯 Exercices

1. **Tester différentes stratégies de diversité** et comparer les gains
2. **Implémenter un système de rejet** basé sur le seuil d'agreement
3. **Optimiser les poids** du weighted voting avec validation croisée
4. **Analyser le coût computationnel** : temps vs gain de performance
5. **Créer un ensemble de CNN** et comparer avec l'ensemble de MLP

---

**Félicitations ! 🎉**

Vous maîtrisez maintenant les **méthodes d'ensemble** et savez comment combiner plusieurs modèles pour obtenir des performances supérieures !